# Test results

With folder `data/cleaned/preds`, we can obtain the results for the position and error metrics but in an accumulated version and not as an average.

In [43]:
import glob, os, pandas as pd, numpy as np, torch, math
from tqdm import tqdm
from BronchoTrack import metrics


def compute_metrics(folder):
    aape, aade = [], []
    for pred_path in tqdm(glob.glob(folder)):
        df = pd.read_csv(pred_path, index_col=0)
        acum = df.cumsum()
        pos_preds = torch.from_numpy(acum.loc[acum.index[-1], ["shift_x", "shift_y", "shift_z"]].values)
        pos_targets = torch.from_numpy(acum.loc[acum.index[-1], ["gt_shift_x", "gt_shift_y", "gt_shift_z"]].values)
        aape.append(metrics.EuclideanDistance.euclidean(pos_preds, pos_targets).numpy())
        rot_targets = acum.loc[acum.index[-1], ["gt_Rx_dif", "gt_Ry_dif", "gt_Rz_dif"]].values
        rot_preds = acum.loc[acum.index[-1], ["Rx_dif", "Ry_dif", "Rz_dif"]].values
        # rot_targets = torch.from_numpy(np.where(rot_targets < 2*math.pi, rot_targets, rot_targets % (2*math.pi)))
        # rot_preds = torch.from_numpy(np.where(rot_preds < 2*math.pi, rot_preds, rot_preds % (2*math.pi)))
        rot_targets = torch.from_numpy(rot_targets)
        rot_preds = torch.from_numpy(rot_preds)
        aade.append(metrics.DirectionError.inverse_cos(rot_preds, rot_targets))
    return "AAPE {} +- {}, AADE {} +- {}".format(np.mean(aape), np.std(aape), np.mean(aade), np.std(aade))

## Example

In [44]:
compute_metrics("data/cleaned/preds/*.csv")

100%|██████████| 2141/2141 [00:07<00:00, 296.69it/s]


'AAPE 1.4075228303417124 +- 0.8117503835730188, AADE 1.55650108924115 +- 0.6851117633183487'